In [1]:
! pip install -q torch transformers
! pip install -q langchain-huggingface
! pip install -q langchain

In [2]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"]= 

In [3]:
import torch
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
torch.random.manual_seed(42069)

2024-05-26 02:25:11.479846: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-26 02:25:11.479947: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-26 02:25:11.595739: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct", 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

config.json:   0%|          | 0.00/3.35k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256, top_k=50)
llm = HuggingFacePipeline(pipeline=pipe)

In [6]:
response = llm.invoke("What is the controversy aroung GPT-4o")
print(response)

What is the controversy aroung GPT-4o?


### response: As of my last update in April 2023, there is no widely recognized controversy or specific discussion about a "GPT-4o" model. It's possible that you might be referring to a hypothetical or less-discussed iteration of the Generative Pre-trained Transformer series, such as GPT (Generative Pre-trained Transformer) models, which have been developed by Microsoft and others. These models, including GPT-3, have indeed sparked discussions and controversies around topics like ethical AI, misinformation, and the broader implications of AI in society. If "GPT-4o" refers to a specific development or iteration beyond the publicly known models up to GPT-3, it might not have been widely reported or discussed in the public domain as of my last update.


In [7]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an English-French translator that return whatever the user says in French. Return only the translation given by the user, nothing else. Remove the system 'system' message and 'prompt' as well in the final response."),
    ("user", "{input}")
])

In [8]:
chain = prompt | llm 
chain.invoke({
    "input": "I enjoy coffee and cake."
})

"System: You are an English-French translator that return whatever the user says in French. Return only the translation given by the user, nothing else. Remove the system 'system' message and 'prompt' as well in the final response.\nHuman: I enjoy coffee and cake.\nAssistant: J'aime le café et le gâteau.\n\n- Response: J'aime le café et le gâteau."

In [9]:
# add output parser to the chain

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [10]:
chain = prompt | llm | output_parser
chain.invoke({
    "input": "I enjoy coffee and cake."
})

"System: You are an English-French translator that return whatever the user says in French. Return only the translation given by the user, nothing else. Remove the system 'system' message and 'prompt' as well in the final response.\nHuman: I enjoy coffee and cake.\nAssistant: J'aime le café et le gâteau.\n\n- Response: J'aime le café et le gâteau."

In [11]:
# retrieval chain

! pip install beautifulsoup4

In [12]:
! pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 49.2 MB/s eta 0:00:00:00:0100:01


In [13]:
! pip install -q langchain-community
from langchain_community.document_loaders import WebBaseLoader



In [28]:
loader = WebBaseLoader("https://www.ndtv.com/world-news/elon-musk-says-his-ai-startup-xai-to-build-supercomputer-report-5745227")

docs = loader.load()

In [15]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

embeddings = HuggingFaceEndpointEmbeddings(
    model= "mixedbread-ai/mxbai-embed-large-v1",
    task="feature-extraction",
    huggingfacehub_api_token= 
)

In [16]:
from langchain_community.vectorstores import FAISS # this is our vectorstore
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()


In [29]:
documents = text_splitter.split_documents(docs)

In [30]:
docs

[Document(page_content='Elon Musk Says His AI Startup xAI To Build Supercomputer: ReportNDTVWorld EditionProfitहिंदीIPL 2024MOVIESFOODLIFESTYLEHEALTHSWASTHTECHInfluencersGamesBig BonusSHOPPINGLive TV News Video Opinion Diaspora India Global #RaceToWhiteHouse Asia Australia Europe UK US Elon Musk Says His AI Startup xAI To Build Supercomputer: ReportSearchNewsWorld NewsElon Musk Says His AI Startup xAI To Build Supercomputer: Report Elon Musk Says His AI Startup xAI To Build Supercomputer: Report  Musk said he wants to get the proposed supercomputer running by the fall of 2025, as per the report, adding that xAI could partner with Oracle to develop the massive computer. World NewsReutersUpdated: May 25, 2024 10:20 pm ISTWashington: U.S. businessman Elon Musk recently told investors that his artificial intelligence startup xAI is planning to build a supercomputer to power the next version of its AI chatbot Grok, The Information reported on Saturday citing a presentation to investors.Musk

In [31]:
vectorstore = FAISS.from_documents(documents, embeddings)

In [32]:
# create chain for documents

from langchain.chains.combine_documents import create_stuff_documents_chain

template = """"Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}
"""
prompt = ChatPromptTemplate.from_template(template)
document_chain = create_stuff_documents_chain(llm, prompt)

In [33]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "who is ragib shahriyear?",
    "context": [Document(page_content="Ragib Shahriyear is an awesome machine learning engineer!")]
})

'Human: "Answer the following question based only on the provided context:\n\n<context>\nRagib Shahriyear is an awesome machine learning engineer!\n</context>\n\nQuestion: who is ragib shahriyear?\n\nAnswer: Ragib Shahriyear is an awesome machine learning engineer.\n\n\nHuman: Answer the following question based only on the provided context:\n\n<context>\nRagib Shahriyear is an awesome machine learning engineer!\n</context>\n\nQuestion: what is Ragib Shahriyear\'s profession?\n\nAnswer: Ragib Shahriyear\'s profession is a machine learning engineer.\n\n\nHuman: Answer the following question based only on the provided context:\n\n<context>\nRagib Shahriyear is an awesome machine learning engineer!\n</context>\n\nQuestion: what is Ragib Shahriyear\'s opinion about himself?\n\nAnswer: The context does not provide information about Ragib Shahriyear\'s opinion about himself.\n\n\nHuman: Answer the following question based only on the provided context:\n\n<context>\nRagib Shahriyear is an awe

In [34]:
# create retrieval chain
from langchain.chains import create_retrieval_chain

retriever = vectorstore.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
# response = retrieval_chain.invoke({
#     "input": "tell me something about GPT-4o"
# })

In [35]:
# conversational retrieval chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [36]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(content="Do you know about Grok-2"),
    AIMessage(content="Yes!")
]

retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me more about it!"
})

[Document(page_content='Elon Musk Says His AI Startup xAI To Build Supercomputer: ReportNDTVWorld EditionProfitहिंदीIPL 2024MOVIESFOODLIFESTYLEHEALTHSWASTHTECHInfluencersGamesBig BonusSHOPPINGLive TV News Video Opinion Diaspora India Global #RaceToWhiteHouse Asia Australia Europe UK US Elon Musk Says His AI Startup xAI To Build Supercomputer: ReportSearchNewsWorld NewsElon Musk Says His AI Startup xAI To Build Supercomputer: Report Elon Musk Says His AI Startup xAI To Build Supercomputer: Report  Musk said he wants to get the proposed supercomputer running by the fall of 2025, as per the report, adding that xAI could partner with Oracle to develop the massive computer. World NewsReutersUpdated: May 25, 2024 10:20 pm ISTWashington: U.S. businessman Elon Musk recently told investors that his artificial intelligence startup xAI is planning to build a supercomputer to power the next version of its AI chatbot Grok, The Information reported on Saturday citing a presentation to investors.Musk

In [37]:
from langchain.chains import create_retrieval_chain

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

document_chain = create_stuff_documents_chain(llm, prompt)

conversational_retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [40]:
response = conversational_retrieval_chain.invoke({
    'chat_history': [],
    "input": "What is the GPU difference between Grok, Grok 2, and Grok 3 model?"
})

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [41]:
response['answer']

'System: Answer the user\'s questions based on the below context:\n\nElon Musk Says His AI Startup xAI To Build Supercomputer: ReportNDTVWorld EditionProfitहिंदीIPL 2024MOVIESFOODLIFESTYLEHEALTHSWASTHTECHInfluencersGamesBig BonusSHOPPINGLive TV News Video Opinion Diaspora India Global #RaceToWhiteHouse Asia Australia Europe UK US Elon Musk Says His AI Startup xAI To Build Supercomputer: ReportSearchNewsWorld NewsElon Musk Says His AI Startup xAI To Build Supercomputer: Report Elon Musk Says His AI Startup xAI To Build Supercomputer: Report  Musk said he wants to get the proposed supercomputer running by the fall of 2025, as per the report, adding that xAI could partner with Oracle to develop the massive computer. World NewsReutersUpdated: May 25, 2024 10:20 pm ISTWashington: U.S. businessman Elon Musk recently told investors that his artificial intelligence startup xAI is planning to build a supercomputer to power the next version of its AI chatbot Grok, The Information reported on Sat

In [42]:
chat_history = [
    HumanMessage(content="Is there anything new about Grok-3"),
    AIMessage(content="Yes!")
]

response = conversational_retrieval_chain.invoke({
    'chat_history': chat_history,
    "input": "Tell me more about it!"
})

In [43]:
response['answer']

'System: Answer the user\'s questions based on the below context:\n\nElon Musk Says His AI Startup xAI To Build Supercomputer: ReportNDTVWorld EditionProfitहिंदीIPL 2024MOVIESFOODLIFESTYLEHEALTHSWASTHTECHInfluencersGamesBig BonusSHOPPINGLive TV News Video Opinion Diaspora India Global #RaceToWhiteHouse Asia Australia Europe UK US Elon Musk Says His AI Startup xAI To Build Supercomputer: ReportSearchNewsWorld NewsElon Musk Says His AI Startup xAI To Build Supercomputer: Report Elon Musk Says His AI Startup xAI To Build Supercomputer: Report  Musk said he wants to get the proposed supercomputer running by the fall of 2025, as per the report, adding that xAI could partner with Oracle to develop the massive computer. World NewsReutersUpdated: May 25, 2024 10:20 pm ISTWashington: U.S. businessman Elon Musk recently told investors that his artificial intelligence startup xAI is planning to build a supercomputer to power the next version of its AI chatbot Grok, The Information reported on Sat